<a href="https://colab.research.google.com/github/PHastings37/Mphys-proj/blob/main/RoryNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To do list
1. Figure out normalization when creating the datasets
2. Get to work with tensorboard to see if weights are changing
3. Issue: currently the network reaches approx minimum loss after just 7 batches of 1 epoch. This may be because the network is very simple or other factors.

In [1]:
!pip install torch torchvision
!pip install opencv-contrib-python
!pip install scikit-learn
!pip install SimpleITK

import numpy as np
import random
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch

from mpl_toolkits.mplot3d import Axes3D
from torch.nn import Module
from torch.nn import Conv3d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from torch import nn
from torch import reshape
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torch.optim import Adam
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

tb = SummaryWriter()

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [4]:
# Connect to GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
# /content/gdrive/MyDrive/MPhys/Data/COLAB-Clinical-Data.csv
# Specify project folder location
project_folder = "/content/gdrive/My Drive/Mphys project/MPhys/Data"
clinical_data_filename = "COLAB-Clinical-Data.csv"
print(os.path.join(project_folder, clinical_data_filename))

Using cuda device
/content/gdrive/My Drive/Mphys project/MPhys/Data/COLAB-Clinical-Data.csv


In [5]:
def equalise_array_lengths(array_1, array_2) :
  """
  This functions takes in the arguments of two lists and makes sure they are returned as the same length.

  Rory Farwell 02/12/2021
  """
  # output_array = []
  if len(array_1) > len(array_2) :
    array_1 = array_1[:len(array_2)]
  elif len(array_1) < len(array_2) :
    array_2 = array_2[:len(array_1)]
  #print(np.vstack((array_1, array_2)))
  # output_array.append(array_1)
  # output_array.append(array_2)
  return (array_1, array_2)

def remove_same_elements(small_array, long_array) :
  """
  For use in the context, all the elements in small_array come from long_array.
  This function will remove all of the elements used in small_array from_long_array.  
  """
  for element in small_array :
    long_array.remove(element)
  return long_array

def create_subgroup(input_array, original_array_length, desired_percentage) :
  """
  This function outputs a subgroup array (e.g. training array) using a specified output array name,
  input array and percentage length
  """
  desired_length = int(original_array_length * desired_percentage)
  output_array = random.sample(input_array, desired_length)
  return output_array
  

# Open the metadata.csv file, convert to an array, and remove column headers
metadata_file = os.path.join(project_folder, clinical_data_filename)
print(f'metadata_file path: {metadata_file}')
metadata = np.genfromtxt(metadata_file, comments = '%', dtype="str", delimiter=",")
print(f"Length of metadata array is {len(metadata)}")

outcome_type = 1 #int(input("Select which outcome you are aiming to predict \n(1=Locoregional, 2=Distant Metastasis, 3=Death):"))
check_day = 3000 #int(input("Select the number of days at which to check for event:"))
which_patients = 1 #int(input("Do you want to include patients whose last follow up is before the check day? (no = 0, yes = 1):"))
patient_with_event = []
patient_no_event = []
outcomes_train = []
outcomes_test = []
images = []

check_day = 365 * 1.5 # This is defining the timeframe for which our CNN will consider the binary output (in days)

patient_IDs = metadata[:,0] # selecting patient IDs from the csv file
time_markers = metadata[:,8] # selecting the day of the last patient review from the csv file
dead_statuses = metadata[:,9] # selecting the dead status on the last review day

time_markers = time_markers.astype(np.float32) # converting to float
dead_statuses = dead_statuses.astype(np.float32) # converting to float


# Empty array which will be appended to later on
# This array will contain the dead statuses of patients on the check day
check_day_dead_statuses = [] 

# sanity check to check progress
counter = 0 

# A counter of howe many patients are dead, alive or censored at the check day
dead_counter = 0
alive_counter = 0
no_info_counter = 0

# Empty arrays that will be appended to.
# They will be appended with the patient IDs and dead statuses on the check day
dead_patient_array = [] 
alive_patient_array = []

# The below 'for' loop will check patient status on the check day and convert if necessary.
# Possibilites are:
# 1) If the patient is dead and the last review point is after than the check day then
# the patient is alive in the check day and so the dead status is changed in the new array
# 2) If the patient is dead and the last review point is before the check day then the patient
# is dead on the check day and their dead status is unchanged
# 3) If the patient is alive and their last review point is after the check day then they
# are alive at the check day and so their dead status remains unchanged
# 4) If the patient has status alive and their last review point is before the 
# check day then we don't know if they are alive at the check day so they are ignored from the list.
for i in range(len(dead_statuses)) :
  temp_dead_status = dead_statuses[i]
  temp_time_marker = time_markers[i]
  if temp_dead_status == 1 : #if the patient is dead
    if temp_time_marker < check_day :
      check_day_dead_statuses.append(1) #confirms that the patient was dead after time 'check_day'
      dead_patient_array.append([patient_IDs[i], 1])
      dead_counter += 1
      continue
    elif temp_time_marker > check_day :
      check_day_dead_statuses.append(0)
      alive_patient_array.append([patient_IDs[i], 0])
      alive_counter += 1
      continue
  elif temp_dead_status == 0 : #if the patient is alive
    if temp_time_marker < check_day :
      no_info_counter += 1
      continue
    elif temp_time_marker > check_day :
      check_day_dead_statuses.append(0)
      alive_patient_array.append([patient_IDs[i], 0])
      alive_counter += 1
      continue

# Printing the results of this 'for' loop (the number of dead and alive patients at the check day)
print(f"Dead counter after {check_day} days: {dead_counter}")
print(f"Alive counter after {check_day} days: {alive_counter}")
print(f"No-info counter after {check_day} days: {no_info_counter}")

# Creating empty arrays that will be appended to later
# These will contain the patient ID and dead status (on the check day).
training_array = []
testing_array = []
validation_array = []

# print(f'Unshuffled dead_patient_array = {dead_patient_array}')
# print(f'Unshuffled alive_patient_array = {alive_patient_array}')

# Shuffle both arrays to ensure a random selection of patient data which will be input to the CNN
random.shuffle(dead_patient_array)
random.shuffle(alive_patient_array)

# print(f'Shuffled dead_patient_array = {dead_patient_array}')
# print(f'Shuffled alive_patient_array = {alive_patient_array}')

# print(equalise_array_lengths(dead_patient_array, alive_patient_array)[0])

# Equalising the length of the 'dead' and 'alive' arrays so that we can ensure optimum training proportions
new_dead_patient_array = equalise_array_lengths(dead_patient_array, alive_patient_array)[0]
new_alive_patient_array = equalise_array_lengths(dead_patient_array, alive_patient_array)[1]
print(f"The alive and dead arrays have been sorted (randomly) so that they are both of length {len(new_dead_patient_array)}, {len(new_alive_patient_array)}")

# print(new_dead_patient_array)
# print(new_alive_patient_array)
# print(len(new_dead_patient_array))
# print(len(new_alive_patient_array))

equalised_array_length = len(new_alive_patient_array)

train_patients_dead = create_subgroup(new_dead_patient_array, equalised_array_length, 0.7)
train_patients_alive = create_subgroup(new_alive_patient_array, equalised_array_length, 0.7)
# print(len(train_patients_dead))
# print(len(train_patients_alive))

new_dead_patient_array = remove_same_elements(train_patients_dead, new_dead_patient_array)
new_alive_patient_array = remove_same_elements(train_patients_alive, new_alive_patient_array)
# print(len(new_dead_patient_array))
# print(len(new_alive_patient_array))

test_patients_dead = create_subgroup(new_dead_patient_array, equalised_array_length, 0.15)
test_patients_alive = create_subgroup(new_alive_patient_array, equalised_array_length, 0.15)
# print(len(test_patients_dead))
# print(len(test_patients_alive))

new_dead_patient_array = remove_same_elements(test_patients_dead, new_dead_patient_array)
new_alive_patient_array = remove_same_elements(test_patients_alive, new_alive_patient_array)
# print(len(new_dead_patient_array))
# print(len(new_alive_patient_array))

validate_patients_dead = create_subgroup(new_dead_patient_array, equalised_array_length, 0.15)
validate_patients_alive = create_subgroup(new_alive_patient_array, equalised_array_length, 0.15)
# print(len(validate_patients_dead))
# print(len(validate_patients_alive))

new_dead_patient_array = remove_same_elements(validate_patients_dead, new_dead_patient_array)
new_alive_patient_array = remove_same_elements(validate_patients_alive, new_alive_patient_array)
# print(len(new_dead_patient_array))
# print(len(new_alive_patient_array))


outcomes_train = train_patients_dead + train_patients_alive
outcomes_test = test_patients_dead + test_patients_alive
outcomes_validate = validate_patients_dead + validate_patients_alive


print(len(outcomes_train))

metadata_file path: /content/gdrive/My Drive/Mphys project/MPhys/Data/COLAB-Clinical-Data.csv
Length of metadata array is 100
Dead counter after 547.5 days: 60
Alive counter after 547.5 days: 40
No-info counter after 547.5 days: 0
The alive and dead arrays have been sorted (randomly) so that they are both of length 40, 40
56


In [6]:
import os
from torchvision.io import read_image

# Normalize class added at 10pm 12/12/2021
class Normalize():
  def __init__(self):
    pass
  
  # def __call__(self, sample):
  #   inputs, targets = sample
  #   inputs = transforms.Normalize(mean = 0.5, std = 0.5)
  #   return inputs, targets
  def __call__(self,vol):
    vol =((vol-(vol.mean()))/vol.std()) + 1
    return vol

# transform = transforms.Compose(
#     [transforms.ToTensor(), transforms.Normalize(mean = 0.5, std = 0.5)] #added at 10:31pm 13/12/2021 to normalize the inputs
# )

transform = transforms.Compose(
    [transforms.ToTensor(), Normalize() ] #added at 11:00pm 13/12/2021 to normalize the inputs. THIS NORMALIZES to mean = 0 and std = -1
)


class ImageDataset(Dataset) :
  def __init__(self, annotations, img_dir, transform = transform, target_transform = None) :
    self.img_labels = annotations
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self) :
    return len(self.img_labels)

  def __getitem__(self,idx) :
    img_path = os.path.join(self.img_dir, self.img_labels[idx][0] + "-GTV-1.nii" )
    image_sitk = sitk.ReadImage(img_path)
    image = sitk.GetArrayFromImage(image_sitk)
    label = self.img_labels[idx][1]
    if self.transform :
      image = self.transform(image)
    if self.target_transform :
      label = self.target_transform(label)
    return image,label

training_data = ImageDataset(outcomes_train, os.path.join(project_folder, "Textured_Masks"), transform = transforms.ToTensor())
validation_data = ImageDataset(outcomes_validate, os.path.join(project_folder, "Textured_Masks"), transform = transforms.ToTensor())
test_data = ImageDataset(outcomes_test, os.path.join(project_folder, "Textured_Masks"), transform = transforms.ToTensor())
print(training_data[0][0][0][0][0])
print(training_data[0][0].mean(), training_data[0][0].std())
print(training_data[0][1])
print(training_data[1][0][0][0][0])
print(training_data[1][0].mean(), training_data[1][0].std())
print(training_data[1][1])
# print(len(training_data))

training_data = ImageDataset(outcomes_train, os.path.join(project_folder, "Textured_Masks"), transform = transform)
validation_data = ImageDataset(outcomes_validate, os.path.join(project_folder, "Textured_Masks"), transform = transform)
test_data = ImageDataset(outcomes_test, os.path.join(project_folder, "Textured_Masks"), transform = transform) 
print(training_data[0][0][0][0][0])
print(training_data[0][0].mean(), training_data[0][0].std())
print(training_data[0][1])
print(training_data[1][0][0][0][0])
print(training_data[1][0].mean(), training_data[1][0].std())
print(training_data[1][1])
# print(len(training_data))



tensor(-1024., dtype=torch.float64)
tensor(-1020.9913, dtype=torch.float64) tensor(54.8529, dtype=torch.float64)
1
tensor(-1024., dtype=torch.float64)
tensor(-1020.7112, dtype=torch.float64) tensor(57.4263, dtype=torch.float64)
1
tensor(0.9452, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64) tensor(1.0000, dtype=torch.float64)
1
tensor(0.9427, dtype=torch.float64)
tensor(1.0000, dtype=torch.float64) tensor(1.0000, dtype=torch.float64)
1


In [7]:
train_dataloader = DataLoader(training_data, batch_size = 4, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 4, shuffle = False)
validation_dataloader = DataLoader(validation_data, batch_size = 4, shuffle = True)

print(len(train_dataloader))

14


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):   
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv3d(1,4,2,2)
        self.pool = nn.MaxPool3d(2,2)
        self.conv2 = nn.Conv3d(4,16,2,2)
        self.conv3 = nn.Conv3d(16,64,2,2)
        self.conv4 = nn.Conv3d(64,256,2,2)
        self.fc1 = nn.Linear(256,64)
        self.fc2 = nn.Linear(64,16)
        self.fc3 = nn.Linear(16,2)

    # Defining the forward pass    
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = self.pool(F.leaky_relu(self.conv3(x)))
        x = self.pool(F.leaky_relu(self.conv4(x)))
        x = x.view(-1, 256)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        # return F.leaky_relu(x)
        return x

# class CNN(nn.Module):   
#     def __init__(self):
#         super(CNN, self).__init__()

#         self.cnn_layers = nn.Sequential(
#             # Defining a 2D convolution layer
#             nn.Conv3d(1, 4, kernel_size=3, stride=3, padding=0),
#             nn.BatchNorm3d(4),
#             nn.LeakyReLU(inplace=True),
#             nn.MaxPool3d(kernel_size=2, stride=2),
#             # Defining another 2D convolution layer
#             nn.Conv3d(4, 16, kernel_size=3, stride=3, padding=0),
#             nn.BatchNorm3d(16),
#             nn.LeakyReLU(inplace=True),
#             nn.MaxPool3d(kernel_size=2, stride=2),
#         )

#         self.linear_layers = nn.Sequential(
#             nn.Linear(5488, 2)
#         )

#     # Defining the forward pass    
#     def forward(self, x):
#         x = self.cnn_layers(x)
#         x = x.view(x.size(0), -1)
#         x = self.linear_layers(x)
#         return x
  
model = CNN().to(device)

In [9]:
from torchsummary import summary
summary(model, (1,264,264,264), batch_size = 4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1      [4, 4, 132, 132, 132]              36
         MaxPool3d-2         [4, 4, 66, 66, 66]               0
            Conv3d-3        [4, 16, 33, 33, 33]             528
         MaxPool3d-4        [4, 16, 16, 16, 16]               0
            Conv3d-5           [4, 64, 8, 8, 8]           8,256
         MaxPool3d-6           [4, 64, 4, 4, 4]               0
            Conv3d-7          [4, 256, 2, 2, 2]         131,328
         MaxPool3d-8          [4, 256, 1, 1, 1]               0
            Linear-9                    [4, 64]          16,448
           Linear-10                    [4, 16]           1,040
           Linear-11                     [4, 2]              34
Total params: 157,670
Trainable params: 157,670
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 280.76
Forwar

In [10]:
# loss and optimizer
learning_rate = 0.0001
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [22]:
#training_loop
num_epochs = 2
n_total_steps = len(train_dataloader)
train_loss = []
valid_loss = []
avg_train_loss = np.empty(0)
avg_valid_loss = np.empty(0)
all_training_losses = []

for epoch in range(num_epochs):
  # ======================================= TRAINING LOOP ======================================
  epoch_train_loss = 0 # will be used for plotting test vs valid loss curves
  n_training_samples = 0

  print(f'Training for epoch {epoch+1}')
  print('=============================================')
  
  model.train()
  for i, (images, labels) in enumerate(train_dataloader):
    # Reformatting input images to have 5 dimensions and casting to a float
    images = reshape(images, (images.shape[0],1 ,264,264,264))
    images = images.float()


    # turning labels of size one to one-hot labels 
    # e.g labels = (1,0,0,1) --> hot_labels [(0,1), (1,0), (1,0), 0,1]
    # This is need because nn.BCELogitsWithLoss() expects inputs of this format
    hot_labels = torch.empty((images.shape[0], 2))
    for index in range(len(labels)):
          if labels[index] == 0 :
            hot_labels[index,0] = 1
            hot_labels[index,1] = 0
          elif labels[index] == 1 :
            hot_labels[index,0] = 0
            hot_labels[index,1] = 1
    
    
    # Send images and one-hot labels to the device (cuda/GPU)
    images = images.to(device)
    hot_labels = hot_labels.to(device)


    # Forward pass
    outputs = model(images)
    # print(outputs)
    loss = criterion(outputs, hot_labels)
    

    # Backwards pass
    optimizer.zero_grad() # Clear gradients before 
    loss.backward()
    optimizer.step()

    # Add the number of images in this batch to n_training_samples which will
    # be used when calculating the average loss per image in the training set
    n_training_samples += labels.shape[0]
    #print(f'Number of samples completed after {i+1} batches = {n_training_samples}')
    

    # Updating the total training loss of this epoch
    # Printing loss for current batch and the new updated total
    # training loss for this epoch
    #print(f'Loss of batch {i+1} = {loss.item():.2f}')
    all_training_losses.append(loss.item())
    epoch_train_loss += loss.item()
    #print(f'Total training loss after batch {i+1} = {epoch_train_loss:.2f}')
    

    # Print a progress statement to ensure the network is running
    if (i+1)%7 == 0 :
      print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
  
  # Append the train_loss list with the total training loss for this epoch
  train_loss.append(epoch_train_loss)

  # Append the avg_train_loss list with the average training loss of this epoch
  avg_train_loss = np.append(avg_train_loss, epoch_train_loss/n_training_samples)
  print(f"avg train loss {avg_train_loss}")

  print(f'Training loss array at end of epoch {epoch + 1}: {train_loss}. Total number of images used = {n_training_samples}')
  print(f'Finished training for epoch {epoch+1}')

  
  
  #================================================ VALIDATION LOOP =================================================
  print(f'Validation for epoch {epoch+1}')
  print('=============================================')
  model.eval()
  with torch.no_grad(): # ensuring gradients are not calculated during the validation loop
    valid_epoch_loss = 0
    n_valid_correct = 0
    n_valid_samples = 0
    for images, labels in validation_dataloader :
      images = images = reshape(images, (images.shape[0],1 ,264,264,264))
      images = images.float()
      hot_labels = torch.empty((images.shape[0], 2))
      #print(new_labels.shape)
      for index in range(len(labels)):
          if labels[index] == 0 :
            hot_labels[index,0] = 1
            hot_labels[index,1] = 0
          elif labels[index] == 1 :
            hot_labels[index,0] = 0
            hot_labels[index,1] = 1
      images = images.to(device)
      hot_labels = hot_labels.to(device)
      outputs = model(images)
      # calculate loss of validation set
      loss = criterion(outputs, hot_labels)
      valid_epoch_loss += loss.item()

      # max returns (value, index) 
      _,predictions = torch.max(outputs, 1)
      _,targets = torch.max(hot_labels, 1)
      #print(f'predictions: {predictions}')
      #print(f'targets: {targets}')
      #print(f'correct in this batch: {(predictions == targets).sum().item()}')
      n_valid_samples += labels.shape[0]
      n_valid_correct += (predictions == targets).sum().item()
      #print(f'n_correct = {n_correct}. n_samples = {n_samples}')
    avg_valid_loss = np.append(avg_valid_loss, valid_epoch_loss/n_valid_samples)
    #valid_loss.append(valid_epoch_loss)
    acc = (100*n_valid_correct)/n_valid_samples
    print(f'Accuracy on validation set for epoch {epoch+1} = {acc:.1f}%')
    print(f'Loss on validation set = {valid_epoch_loss}')

    print(f'Finished validation for epoch {epoch+1}')
    print('=============================================')

    #================================================ TENSORBOARD STUFF ===============================================
  

  tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
  tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
  tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
  tb.add_histogram("conv2.weight", model.conv2.weight, epoch)
  tb.add_histogram("conv3.bias", model.conv3.bias, epoch)
  tb.add_histogram("conv3.weight", model.conv3.weight, epoch)
  tb.add_histogram("conv4.bias", model.conv4.bias, epoch)
  tb.add_histogram("conv4.weight", model.conv4.weight, epoch)
  tb.add_histogram("fc1.bias", model.fc1.bias, epoch)
  tb.add_histogram("fc1.weight", model.fc1.weight, epoch)
  tb.add_histogram("fc2.bias", model.fc2.bias, epoch)
  tb.add_histogram("fc2.weight", model.fc2.weight, epoch)
  tb.add_histogram("fc3.bias", model.fc3.bias, epoch)
  tb.add_histogram("fc3.weight", model.fc3.weight, epoch)
  tb.add_scalar("training_loss", avg_train_loss[epoch], epoch)
  tb.add_scalar("validation_loss", avg_valid_loss[epoch], epoch)

    # tb.add_scalar("Validation loss", (valid_epoch_loss/n_valid_samples), epoch)

# close tensorboard once the code has training loop has finished running
# tb.close() 

print('FINISHED TRAINING')
print(f'All training batch losses = {all_training_losses}')
print(f'Training losses = {train_loss}')
print(f'Average training losses = {avg_train_loss}')
print(f'Validation losses = {avg_valid_loss}')
#testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_dataloader :
      images = images = reshape(images, (images.shape[0],1 ,264,264,264))
      images = images.float()
      hot_labels = torch.empty((images.shape[0], 2))
      #print(new_labels.shape)
      for index in range(len(labels)):
          if labels[index] == 0 :
            hot_labels[index,0] = 1
            hot_labels[index,1] = 0
          elif labels[index] == 1 :
            hot_labels[index,0] = 0
            hot_labels[index,1] = 1
      images = images.to(device)
      hot_labels = hot_labels.to(device)
      outputs = model(images)
      # max returns (value, index) 
      _,predictions = torch.max(outputs, 1)
      _,targets = torch.max(hot_labels,1)
      #print(f'predictions: {predictions}')
      #print(f'targets: {targets}')
      n_samples += hot_labels.shape[0]
      n_correct += (predictions == targets).sum().item()
      #print(f'n_correct = {n_correct}. n_samples = {n_samples}')
    
    acc = (100*n_correct)/n_samples
    print(f'Accuracy on testing set = {acc:.1f}%')


# time renewal
tb.close()

Training for epoch 1
Epoch 1/2, step 7/14, loss = 0.6933
Epoch 1/2, step 14/14, loss = 0.5785
avg train loss [0.162954]
Training loss array at end of epoch 1: [9.125423729419708]. Total number of images used = 56
Finished training for epoch 1
Validation for epoch 1
Accuracy on validation set for epoch 1 = 83.3%
Loss on validation set = 1.8358817100524902
Finished validation for epoch 1
Training for epoch 2
Epoch 2/2, step 7/14, loss = 0.7014
Epoch 2/2, step 14/14, loss = 0.6124
avg train loss [0.162954  0.1622121]
Training loss array at end of epoch 2: [9.125423729419708, 9.083877801895142]. Total number of images used = 56
Finished training for epoch 2
Validation for epoch 2
Accuracy on validation set for epoch 2 = 83.3%
Loss on validation set = 1.81387859582901
Finished validation for epoch 2
FINISHED TRAINING
All training batch losses = [0.6459228992462158, 0.6631618738174438, 0.6436505317687988, 0.6304388046264648, 0.7168189287185669, 0.6671513319015503, 0.6932774782180786, 0.58782

In [ ]:
avg_valid_loss = []
for i in range(len(valid_loss)):
  avg_valid_loss.append(valid_loss[i]/n_valid_samples)

print(n_valid_samples)
print(avg_valid_loss)

In [ ]:
import matplotlib.pyplot as plt

epochs = np.array(range(len(avg_train_loss))) + 1
avg_train_loss_numpy = np.array(avg_train_loss)
avg_valid_loss_numpy = np.array(avg_valid_loss)
print(epochs)

fig = plt.figure()
plt.plot(epochs,  avg_train_loss_numpy, label = 'Average training loss')
plt.plot(epochs, avg_valid_loss_numpy, label = 'Average validation loss')
#plt.title('Average losses: training vs validation')
plt.ylabel('Average loss')
plt.xlabel('Epoch number')
plt.title(f'Losses over {len(epochs)} epochs with learning rate = {learning_rate}')
plt.legend(loc='best')
plt.show()


In [ ]:
data = next(iter(test_dataloader))
# print(data[0][0].numpy())
array = data[0][0].numpy()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x,y,z = np.where(array > array[0][0][0])
ax.scatter(x, y, z, c=z, alpha=1)
print(len(array))

ax.set_xlim(0,len(array))
ax.set_ylim(0,len(array))
ax.set_zlim(0,len(array))